### Notebook to import the dataset, pre-process it, split it into train and test and balance the train

1. Open the dataset

In [3]:
import pandas as pd
import re
import numpy as np

In [4]:
df = pd.read_csv('C:\\Users\\danie\\PycharmProjects\\NLP_project\\data\\quora_duplicate_questions.tsv', sep='\t', index_col =0, header=0)

In [5]:
df

,qid1,qid2,question1,question2,is_duplicate
id,,,,,
0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...,...,...
404285,433578,379845,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404286,18840,155606,Do you believe there is life after death?,Is it true that there is life after death?,1
404287,537928,537929,What is one coin?,What's this coin?,0


2. Remove indices which end up empty after the remove_words preprocessing

In [6]:
q1_empty_ind = [3306, 39405, 108978, 109311, 115347, 151922, 158778, 189396, 190570, 199110, 290090, 301583, 325530, 326297, 328601, 329933, 381124]
q2_empty_ind = [20072, 144343, 181695, 198913, 384293, 400153]
remove_ind = q1_empty_ind +q2_empty_ind
len(remove_ind)

23

In [7]:
df = df.drop(remove_ind, axis =0)

In [8]:
df

,qid1,qid2,question1,question2,is_duplicate
id,,,,,
0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...,...,...
404285,433578,379845,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404286,18840,155606,Do you believe there is life after death?,Is it true that there is life after death?,1
404287,537928,537929,What is one coin?,What's this coin?,0


In [128]:
#Split into question 1 and question 2 and then run the remove_words

In [9]:
question1 = df['question1']
question1 = question1.str.replace('\\n', ' ', regex= True)

In [10]:
np.savetxt('C:\\Users\\danie\\PycharmProjects\\NLP_project\\data\\corpus\\question1.txt', question1.values, fmt='%s', encoding = 'utf-8', delimiter ='\n')

In [11]:
question2 = df['question2']
question2 = question2.str.replace('\\n', ' ', regex= True)

In [12]:
np.savetxt('C:\\Users\\danie\\PycharmProjects\\NLP_project\\data\\corpus\\question2.txt', question2.values, fmt = '%s', encoding = 'utf-8', delimiter ='\n')

Extracted and saved question 1 and question 2 of the dataframe. Will now replace in the dataframe the original with the cleaned one

In [13]:
question1_clean = pd.read_csv('C:\\Users\\danie\\PycharmProjects\\NLP_project\\data\\corpus\\question1.clean.txt', sep="\n", names=['Clean'])
question2_clean = pd.read_csv('C:\\Users\\danie\\PycharmProjects\\NLP_project\\data\\corpus\\question2.clean.txt', sep="\n", names=['Clean'])

We are getting some NaN cells which we remove from both questions

In [14]:
df['question1_clean'] = question1_clean.values
df['question2_clean'] = question2_clean.values

In [15]:
null_ind = df.isnull()

In [16]:
null_val1 = null_ind[null_ind['question1_clean']==True]

In [17]:
null_val2 = null_ind[null_ind['question2_clean']==True]

In [18]:
null_val = pd.concat([null_val1, null_val2])

In [19]:
null_val

,qid1,qid2,question1,question2,is_duplicate,question1_clean,question2_clean
id,,,,,,,
363362,False,False,True,False,False,True,False
105780,False,False,False,True,False,False,True
141281,False,False,False,False,False,False,True
201841,False,False,False,True,False,False,True
205947,False,False,False,False,False,False,True


In [20]:
df = df.drop(null_val.index)

In [21]:
df = df.drop(columns = ['qid1', 'qid2'])

In [22]:
def additional_cleaning(text):
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    return text

In [23]:
df['question1_clean_clean']= df.apply(lambda row: additional_cleaning(row['question1_clean']), axis=1)
df['question2_clean_clean']= df.apply(lambda row: additional_cleaning(row['question2_clean']), axis=1)


In [24]:
df['tokenized_sents1'] = df.apply(lambda row: len(re.findall(r'\w+', row['question1_clean_clean'])), axis=1)
df['tokenized_sents2'] = df.apply(lambda row: len(re.findall(r'\w+', row['question2_clean_clean'])), axis=1)

In [25]:
df['tokenized_sents1'][13016]

0

In [26]:
empty_1 = list(df[df['tokenized_sents1'] == 0].index)
empty_2 = list(df[df['tokenized_sents2'] == 0].index)
empty = empty_1 +empty_2

In [27]:
df = df.drop(empty)

Import the quora train test split according to literature and match the cleaned dataframe to that one

In [28]:
def open_in_pd(path, file_name):
    df =  pd.read_csv(path+file_name+'.tsv', sep='\t', header=None)
    df.columns = ['label','question1','question2','index']
    return df

In [29]:
train = open_in_pd('C:\\Users\\danie\\PycharmProjects\\NLP_project\\data\\Quora_question_pair_partition\\', 'train')
test = open_in_pd('C:\\Users\\danie\\PycharmProjects\\NLP_project\\data\\Quora_question_pair_partition\\', 'test')
dev = open_in_pd('C:\\Users\\danie\\PycharmProjects\\NLP_project\\data\\Quora_question_pair_partition\\', 'dev')

In [30]:
len(train)

384348

In [31]:
len(dev)

10000

In [32]:
len(test)

10000

In [33]:
train = pd.concat([train, dev])


In [34]:
train_ind = train['index'].tolist()
len(train_ind)

394348

In [35]:
train.sort_values(by=['index'])


,label,question1,question2,index
255608,0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
233001,0,What is the story of Kohinoor -LRB- Koh-i-Noor...,What would happen if the Indian government sto...,1
194764,0,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,2
195499,0,Why am I mentally very lonely ? How can I solv...,Find the remainder when -LSB- math -RSB- 23 ^ ...,3
216934,0,"Which one dissolve in water quikly sugar , sal...",Which fish would survive in salt water ?,4
...,...,...,...,...
12343,0,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,404346
151400,1,Do you believe there is life after death ?,Is it true that there is life after death ?,404347
352700,0,What is one coin ?,What 's this coin ?,404348
32895,0,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,404349


In [36]:
test_ind = test['index'].tolist()
len(test_ind)

10000

In [37]:
test_train_ind = train_ind+test_ind
len(test_train_ind)

404348

In [38]:
df_match = df[df.index.isin(test_train_ind)]
df_match

,question1,question2,is_duplicate,question1_clean,question2_clean,question1_clean_clean,question2_clean_clean,tokenized_sents1,tokenized_sents2
id,,,,,,,,,
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,step step guide invest share market india \?,step step guide invest share market \?,step step guide invest share market india,step step guide invest share market,7,6
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,story kohinoor \( koh noor \) diamond \?,would happen indian government stole kohinoor ...,story kohinoor koh noor diamond,would happen indian government stole kohinoor ...,5,10
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,increase speed internet connection using vpn \?,internet speed increased hacking dns \?,increase speed internet connection using vpn,internet speed increased hacking dns,6,5
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,mentally lonely \? solve \?,"find remainder math 23 24 math divided 24 , 23 \?",mentally lonely solve,find remainder math 23 24 math divided 24 23,3,9
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,"one dissolve water quikly sugar , salt , metha...",fish would survive salt water \?,one dissolve water quikly sugar salt methane c...,fish would survive salt water,10,5
...,...,...,...,...,...,...,...,...,...
404285,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0,many keywords racket programming language late...,many keywords perl programming language latest...,many keywords racket programming language late...,many keywords perl programming language latest...,7,7
404286,Do you believe there is life after death?,Is it true that there is life after death?,1,believe life death \?,true life death \?,believe life death,true life death,3,3
404287,What is one coin?,What's this coin?,0,one coin \?,'s coin \?,one coin,coin,2,1


In [39]:
df_train = df[df.index.isin(train_ind)]

Only keep sentences shorter than 30 tokens in the training set

In [40]:
short_q1 = df_train.loc[df['tokenized_sents1']>30]
short_q2 = df_train.loc[df['tokenized_sents2']>30]
short_sents = short_q1.index.tolist() +short_q2.index.tolist()
len(short_sents)

233

In [41]:
df_train = df_train.drop(short_sents)

In [42]:
df_train_ind = df_train[['is_duplicate']]
df_train_ind['train_test'] = 'train'
df_train_ind.columns = ['label', 'train_test']
df_train_ind['index'] = df_train_ind.index
df_train_ind = df_train_ind[['index','label', 'train_test']]
df_train_ind

C:\Users\danie\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\danie\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,index,label,train_test
id,,,
0,0,0,train
1,1,0,train
2,2,0,train
3,3,0,train
4,4,0,train
...,...,...,...
404284,404284,1,train
404285,404285,0,train
404287,404287,0,train


Balance the training 

In [43]:
df_train_ind[df_train_ind['train_test']=='train']['label'].value_counts()

0    248452
1    145470
Name: label, dtype: int64

In [44]:
majority = df_train_ind.loc[df_train_ind['label']==0].count()[0]
minority = df_train_ind.loc[df_train_ind['label']==1].count()[0]

In [45]:
np.random.seed(11)
remove_n = majority - minority
drop_indices = np.random.choice(df_train_ind[df_train_ind['label']==0].index, remove_n, replace=False)
df_train_ind_balanced = df_train_ind.drop(drop_indices)
df_train_ind_balanced['label'].value_counts()

1    145470
0    145470
Name: label, dtype: int64

In [46]:
df_test = df[df.index.isin(test_ind)]
df_test_ind = df_test[['is_duplicate']]
df_test_ind['train_test'] = 'test'
df_test_ind.columns = ['label', 'train_test']
df_test_ind['index'] = df_test_ind.index
df_test_ind = df_test_ind[['index','label', 'train_test']]
df_test_ind

C:\Users\danie\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\danie\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,index,label,train_test
id,,,
52,52,0,test
57,57,0,test
113,113,1,test
130,130,0,test
173,173,1,test
...,...,...,...
404156,404156,0,test
404200,404200,1,test
404251,404251,0,test


In [47]:
df_test_ind['label'].value_counts()

0    6210
1    3784
Name: label, dtype: int64

Split test set into short, medium and long sentences

In [48]:
df_test = df_test.reset_index()

In [49]:
short_q1_test = df_test[df_test['tokenized_sents1']<6]
short_q2_test = df_test[df_test['tokenized_sents2']<6]
short_sents_test = list(set(short_q1_test.index.tolist()) & set(short_q2_test.index.tolist()))
short_sents_test = sorted(short_sents_test)
len(short_sents_test)

with open('short_sents.txt', 'w') as f:
    for item in short_sents_test:
        f.write("%s\n" % item)

In [50]:
medium_q1_test = df_test[(df_test['tokenized_sents1']<16) & (df_test['tokenized_sents1']>5)]
medium_q2_test = df_test[(df_test['tokenized_sents2']<16) & (df_test['tokenized_sents2']>5)]
medium_sents_test = list(set(medium_q1_test.index.tolist()) & set(medium_q2_test.index.tolist()))
medium_sents_test = sorted(medium_sents_test)
len(medium_sents_test)

with open('medium_sents.txt', 'w') as f:
    for item in medium_sents_test:
        f.write("%s\n" % item)

In [51]:
long_q1_test = df_test[df_test['tokenized_sents1']>15]
long_q2_test = df_test[df_test['tokenized_sents2']>15]
long_sents_test = list(set(long_q1_test.index.tolist()) & set(long_q2_test.index.tolist()))
long_sents_test = sorted(long_sents_test)
len(long_sents_test)

with open('long_sents.txt', 'w') as f:
    for item in long_sents_test:
        f.write("%s\n" % item)

In [85]:
len(medium_sents_test)

2399

In [53]:
match_len_sents = short_sents_test + medium_sents_test +long_sents_test

In [54]:
dif_index = list(set(df_test.index.tolist()).symmetric_difference(match_len_sents))

with open('unmatched_len.txt', 'w') as f:
    for item in dif_index:
        f.write("%s\n" % item)

In [55]:
len(short_sents_test)+len(medium_sents_test)+len(long_sents_test)

7176

In [56]:
quora_train_test_split = pd.concat([df_train_ind_balanced,df_test_ind])
quora_train_test_split_sorted = quora_train_test_split.sort_values(['index'])
quora_train_test_split_sorted

,index,label,train_test
id,,,
0,0,0,train
1,1,0,train
2,2,0,train
3,3,0,train
5,5,1,train
...,...,...,...
404284,404284,1,train
404285,404285,0,train
404286,404286,1,test


In [57]:
quora_train_test_split_sorted['train_test'].value_counts()

train    290940
test       9994
Name: train_test, dtype: int64

In [58]:
quora_train_test_split_sorted[quora_train_test_split_sorted['train_test']=='train']['label'].value_counts()

1    145470
0    145470
Name: label, dtype: int64

In [59]:
np.random.seed(11)
remove_n = 190000
drop_indices1 = np.random.choice(quora_train_test_split_sorted[quora_train_test_split_sorted['train_test']=='train'].index, remove_n, replace=False)
quora_train_test_split_subset = quora_train_test_split_sorted.drop(drop_indices1)

Subsample the dataset

In [60]:
quora_train_test_split_subset[quora_train_test_split_subset['train_test']=='train']['label'].value_counts()

1    50673
0    50267
Name: label, dtype: int64

Need to also remove the droped indices from the cleaned questions

In [61]:
df_final = df[df.index.isin(quora_train_test_split_subset.index)]

In [62]:
df_final

,question1,question2,is_duplicate,question1_clean,question2_clean,question1_clean_clean,question2_clean_clean,tokenized_sents1,tokenized_sents2
id,,,,,,,,,
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,step step guide invest share market india \?,step step guide invest share market \?,step step guide invest share market india,step step guide invest share market,7,6
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,story kohinoor \( koh noor \) diamond \?,would happen indian government stole kohinoor ...,story kohinoor koh noor diamond,would happen indian government stole kohinoor ...,5,10
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,increase speed internet connection using vpn \?,internet speed increased hacking dns \?,increase speed internet connection using vpn,internet speed increased hacking dns,6,5
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,mentally lonely \? solve \?,"find remainder math 23 24 math divided 24 , 23 \?",mentally lonely solve,find remainder math 23 24 math divided 24 23,3,9
6,Should I buy tiago?,What keeps childern active and far from phone ...,0,buy tiago \?,keeps childern active far phone video games \?,buy tiago,keeps childern active far phone video games,2,7
...,...,...,...,...,...,...,...,...,...
404277,What does analytics do?,What are analytical people like?,0,analytics \?,analytical people like \?,analytics,analytical people like,1,3
404280,What are some outfit ideas to wear to a frat p...,What are some outfit ideas wear to a frat them...,1,outfit ideas wear frat party \?,outfit ideas wear frat themed party \?,outfit ideas wear frat party,outfit ideas wear frat themed party,5,6
404286,Do you believe there is life after death?,Is it true that there is life after death?,1,believe life death \?,true life death \?,believe life death,true life death,3,3


In [63]:
question1 = df_final['question1_clean_clean']
question1_final_bal_sub = question1.str.replace('\\n', ' ', regex= True)

In [64]:
question2 = df_final['question2_clean_clean']
question2_final_bal_sub = question2.str.replace('\\n', ' ', regex= True)

In [65]:
np.savetxt('C:\\Users\\danie\\PycharmProjects\\NLP_project\\data\\corpus\\question1_sub_bal.clean.txt', question1_final_bal_sub.values, fmt = '%s', encoding = 'utf-8', delimiter ='\n')
np.savetxt('C:\\Users\\danie\\PycharmProjects\\NLP_project\\data\\corpus\\question2_sub_bal.clean.txt', question2_final_bal_sub.values, fmt = '%s', encoding = 'utf-8', delimiter ='\n')

In [66]:
np.savetxt('C:\\Users\\danie\\PycharmProjects\\NLP_project\\data\\quora_split_balanced_subset.txt', quora_train_test_split_subset.values, fmt='%s', delimiter='\t')

#### Check where the DNN and GNN differ in their classification - i.e. whether one is better on short or long sentences for exmaple

In [67]:
indices_dnn_right = np.loadtxt('indices_dnn_right.txt')
indices_dnn_right = indices_dnn_right[:-1]
indices_gnn_right = np.loadtxt('indices_gnn_right.txt')

In [68]:
print('GNN: mean sentence 1 length:' + str(df_test.iloc[indices_gnn_right]['tokenized_sents1'].mean()))
print('GNN: variance sentence 1 length:' + str(np.var(df_test.iloc[indices_gnn_right]['tokenized_sents1'])))
print('GNN: mean sentence 2 length:' + str(df_test.iloc[indices_gnn_right]['tokenized_sents2'].mean()))
print('GNN: variance sentence 2 length:' + str(np.var(df_test.iloc[indices_gnn_right]['tokenized_sents2'])))
print('GNN: mean sentence 1 and sentence 2 length difference:' + str(abs(df_test.iloc[indices_gnn_right]['tokenized_sents1']-(df_test.iloc[indices_gnn_right]['tokenized_sents2'])).mean()))
print('GNN: variance sentence 1 and sentence 2 length difference:' + str(np.var(np.absolute(df_test.iloc[indices_gnn_right]['tokenized_sents1']-df_test.iloc[indices_gnn_right]['tokenized_sents2']))))
print('-'*60)
print('DNN: mean sentence 1 length:' + str(df_test.iloc[indices_dnn_right]['tokenized_sents1'].mean()))
print('DNN: variance sentence 1 length:' + str(np.var(df_test.iloc[indices_dnn_right]['tokenized_sents1'])))
print('DNN: mean sentence 2 length:' + str(df_test.iloc[indices_dnn_right]['tokenized_sents2'].mean()))
print('DNN: variance sentence 2 length:' + str(np.var(df_test.iloc[indices_dnn_right]['tokenized_sents2'])))
print('DNN: mean sentence 1 and sentence 2 length difference:' + str(abs(df_test.iloc[indices_dnn_right]['tokenized_sents1']-df_test.iloc[indices_dnn_right]['tokenized_sents2']).mean()))
print('DNN: variance sentence 1 and sentence 2 length difference:' + str(np.var(np.absolute(df_test.iloc[indices_dnn_right]['tokenized_sents1']-df_test.iloc[indices_dnn_right]['tokenized_sents2']))))
print('-'*60)

GNN: mean sentence 1 length:5.4747583243823845
GNN: variance sentence 1 length:9.160211407758322
GNN: mean sentence 2 length:5.564983888292159
GNN: variance sentence 2 length:10.142662164079793
GNN: mean sentence 1 and sentence 2 length difference:1.8968850698174007
GNN: variance sentence 1 and sentence 2 length difference:6.257895775190586
------------------------------------------------------------
DNN: mean sentence 1 length:5.664265129682997
DNN: variance sentence 1 length:10.297944921060758
DNN: mean sentence 2 length:5.739193083573487
DNN: variance sentence 2 length:11.469443729289315
DNN: mean sentence 1 and sentence 2 length difference:1.9164265129682998
DNN: variance sentence 1 and sentence 2 length difference:6.583793570248045
------------------------------------------------------------


In [69]:
from collections import Counter 
  
def most_frequent(List): 
    occurence_count = Counter(List) 
    return occurence_count.most_common(1)[0][0] 
    

print(most_frequent(df_test.iloc[indices_dnn_right]['tokenized_sents2'])) 

5


In [70]:
indices_inductive_15 = np.loadtxt('inductive_ind_15.txt')


In [71]:
print('mean sentence 1 length:' + str(df_test.iloc[indices_inductive_15]['tokenized_sents1'].mean()))
print('variance sentence 1 length:' + str(np.var(df_test.iloc[indices_inductive_15]['tokenized_sents1'])))
print('mean sentence 2 length:' + str(df_test.iloc[indices_inductive_15]['tokenized_sents2'].mean()))
print('variance sentence 2 length:' + str(np.var(df_test.iloc[indices_inductive_15]['tokenized_sents2'])))

print('-'*60)


mean sentence 1 length:5.696312364425163
variance sentence 1 length:9.525995078133453
mean sentence 2 length:5.5249457700650755
variance sentence 2 length:9.017273587080815
------------------------------------------------------------


In [76]:
print('min train sentence length:' +str(df_train['tokenized_sents1'].min()))
print('min train sentence length:' +str(df_train['tokenized_sents2'].min()))
print('min test sentence length:' +str(df_test['tokenized_sents1'].min()))
print('min test sentence length:' +str(df_test['tokenized_sents2'].min()))

min train sentence length:1
min train sentence length:1
min test sentence length:1
min test sentence length:1


In [75]:
print('mean train sentence length:' +str(df_train['tokenized_sents1'].mean()))
print('mean train sentence length:' +str(df_train['tokenized_sents2'].mean()))
print('mean test sentence length:' +str(df_test['tokenized_sents1'].mean()))
print('mean test sentence length:' +str(df_test['tokenized_sents2'].mean()))

mean train sentence length:5.5879742690177245
mean train sentence length:5.6559725021704805
mean test sentence length:5.6039623774264555
mean test sentence length:5.685311186712028


In [77]:
print('max train sentence length:' +str(df_train['tokenized_sents1'].max()))
print('max train sentence length:' +str(df_train['tokenized_sents2'].max()))
print('max test sentence length:' +str(df_test['tokenized_sents1'].max()))
print('max test sentence length:' +str(df_test['tokenized_sents2'].max()))

max train sentence length:30
max train sentence length:30
max test sentence length:31
max test sentence length:44
